In [2]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Exams Timetable')


Restricted license - for non-production use only - expires 2026-11-23


In [4]:
def exam_length(c):
    # return data.exam_lengths[c] 
    return 6 #(1.5hr)

print(data.n_courses)
exam_spans = {c: data.begin_end_pairs(data.get_exam_duration(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)


10


In [5]:
C = tt.addVars(
    [
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    name="C",
)

In [ ]:
# Hard constraints

# Exactly one exam_slot for each course
# sum of enrolments in courses assigned to a room <= sum of capacities of rooms allocated
# no clash for any student

In [1]:
def signum(x):
    if x>0:
        return 1
    return 0

In [ ]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course",
)

# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

# tt.addConstrs(
   
#     # (
#     #     C['*', r] * E.sum('*', '*', '*', '*') <= data.room_capacities[r] 
#     #     for r in range(data.n_rooms)
#     # ),
# )
tt.addConstrs((gp.quicksum(C[c,r]*S[b,e,d,s]
                          for c in range(data.n_courses)
                          for b,e in covering_sessions(s, t)) <= 1
              for d in range(data.n_days)
              for r in range(data.n_rooms)
              for t in range(data.n_times)),
              name = 'room_capacity_constraint'
             )

# Constraint 3: All students for a course should write the exam 
tt.addConstrs(
    # (
    #     C[c, r] * E[c, b, e, d] <= data.enrolments[c]
    #     for c in range(data.n_courses)
    #     for r in range(data.n_rooms)
    #     for b, e in exam_spans[c]
    #     for d in range(data.n_days)
    # ),
    (
        C[c, '*'] * E.sum(c, '*', '*', '*') <= data.enrolments[c]
        for c in range(data.n_courses)
    ),
    name="course_enrollment_constraint",
)

# Constraint 4: Student clash constraint

# for each student, for each day, for each time slot, at most one exam
# sigma Ecbed <= 1 for courses enrolled by student s

# tt.addConstrs((gp.quicksum(C[c,r]*S[b,e,d,s]
#                           for c in range(data.n_courses)
#                           for r in range(data.n_rooms)
#                           for s in classes_for_course(c)
#                           for b,e in covering_sessions(s, t)) <= 1
#               for d in range(data.n_days)
#               for t in range(data.n_times)),
#               name='room_double_booking'
#              )

tt.addConstrs(
    (
        gp.quicksum(E[c, '*', '*', '*'] for c in data.student_courses[s]) <= 1
        for s in range(data.n_students)
    ),
   
    name="student_clash_constraint",
)

# No course can have more than 3 rooms alloted
tt.addConstrs(
    (
        gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)





